In [77]:
import numpy as np
import chainer
from chainer.backends import cuda
from chainer import Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
import random

from opyenxes.model.XLog import XLog
from opyenxes.data_in.XUniversalParser import XUniversalParser
from opyenxes.classification.XEventAttributeClassifier import XEventAttributeClassifier

In [4]:
bpic_2011_path = "../logs/bpic2011.xes"

with open(bpic_2011_path) as bpic2011_file:
    bpic2011_rlog = XUniversalParser().parse(bpic2011_file)

bpic2011_log = bpic2011_rlog[0] # the rest of this array is empty anyway as len(bpic2011_rlog) == 1

Unknown extension: http://www.xes-standard.org/meta_time.xesext
Unknown extension: http://www.xes-standard.org/meta_life.xesext
Unknown extension: http://www.xes-standard.org/meta_org.xesext
Unknown extension: http://www.xes-standard.org/meta_concept.xesext
Unknown extension: http://www.xes-standard.org/meta_3TU.xesext
Unknown extension: http://www.xes-standard.org/meta_general.xesext


In [72]:
event_traces = [[ ev.get_attributes()["concept:name"].get_value() for ev in trace ] for trace in bpic2011_log ]
event_traces = [ ['<bos>'] + l + ['<eos>'] for l in event_traces ]

In [53]:
class EventDataset(chainer.dataset.DatasetMixin):
    def __init__(self, event_traces, windowsize):
        self.data = [ w for w in self.window_features(event_traces, windowsize) ]
    
    def window_features(self,traces,windowsize):
        for trace in traces:
            for event_i in range(0, len(trace)-windowsize+1):
                yield(trace[event_i:event_i+windowsize])
                
    def __len__(self):
        return len(self.data)
    
    def get_example(self, i):
        example = self.data[i]
        return tuple(example)

In [83]:
random.shuffle(event_traces)
test_traces  = event_traces[:int(.8*len(event_traces))]
train_traces = event_traces[int(.8*len(event_traces)):]
test  = EventDataset(event_traces, 4)
train = EventDataset(event_traces, 4)

In [84]:
class RNN(Chain):
    def __init__(self):
        super(RNN, self).__init__()
        with self.init_scope():
            self.embed = L.EmbedID(1000, 100)  # word embedding
            self.mid = L.LSTM(100, 50)  # the first LSTM layer
            self.out = L.Linear(50, 1000)  # the feed-forward output layer

    def reset_state(self):
        self.mid.reset_state()

    def __call__(self, cur_word):
        # Given the current word ID, predict the next word.
        x = self.embed(cur_word)
        h = self.mid(x)
        y = self.out(h)
        return y

rnn = RNN()
model = L.Classifier(rnn)
optimizer = optimizers.SGD()
optimizer.setup(model)